In [ ]:
# %matplotlib widget
import pickle
import numpy as np

method = "LN_COBYLA (NLopt)"
problem = "maxcut"
p = 1
n = 12
seed_pool = list(range(1000))

data = pickle.load(
    open(
        f"data/{problem}/configs/max_ar/LN_COBYLA (NLopt)-p{p}-q{n}-s0-999.pckl",
        "rb",
    ),
)
max_ar = data["result"][:len(seed_pool)]

data = pickle.load(
    open(
        f"data/{problem}/configs/budget/{method}-p{p}-q{n}-s{seed_pool[0]}-{seed_pool[-1]}.pckl",
        "rb",
    ),
)
config = data["config"]
result = data["result"][:,:,:]
initial_ar = np.array(data["initial_ar"])

print(config, result.shape)
shots_pool = [d["shots"] for d in config["executor_kwargs"]][:]
maxfev_pool = config["maxeval"][:]


In [ ]:
from evaluate_energy import get_evaluate_energy, load_problem
from qokit.parameter_utils import get_fixed_gamma_beta
import matplotlib.pyplot as plt
from scipy.spatial import distance

# seed_pool = [596, 617, 539, 972, 816, 0, 1, 2, 3, 4]
data = []
for seed in seed_pool:
    instance, precomputed_energies = load_problem(problem, n, seed)
    eval_func = get_evaluate_energy(
        instance,
        precomputed_energies,
        p,
        objective="expectation",
        simulator="c",
    )
    gamma, beta = get_fixed_gamma_beta(3, p)
    gamma = gamma[0]
    beta = beta[0] * 4
    gamma_diff = (eval_func([gamma-1.5, beta]) - eval_func([gamma, beta]))
    beta_diff = (eval_func([gamma, beta-1.5]) - eval_func([gamma, beta]))
    data.append(gamma_diff / beta_diff)
    print(data[-1], gamma_diff, beta_diff, (max_ar - initial_ar)[seed])
    # data.append(distance.minkowski([instance.edges[e]['weight'] for e in instance.edges], np.ones(instance.number_of_edges())))

plt.scatter((max_ar - initial_ar).flat, np.abs(np.array(data) - data[596]))
# plt.scatter((max_ar - initial_ar).flat, np.abs(data))
# plt.ylim(0, 1)